In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import sys
sys.path.append('../')
import src.cleaning_functions as cf
import pyjsonviewer
from functools import reduce
import operator
from pymongo import MongoClient
from pymongo import GEOSPHERE
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from bson.json_util import dumps

In [2]:
load_dotenv()

True

#### Utilizo los token de la API de Foursquare para poder llamarla 

In [3]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/search'
url_recomendados = 'https://api.foursquare.com/v2/venues/explore'

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
***30% of the company staff have at least 1 child.**
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
***Account managers need to travel a lot.**
***Everyone in the company is between 25 and 40, give them some place to go party.**
***The CEO is vegan.**
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

## MADRID

#### Utilizo una función para buscar las coordenadas del punto concreto elegido para la ciudad de Madrid

In [5]:
cf.geocode("Castellana 81, Paseo de la Castellana, Madrid")

{'type': 'Point', 'coordinates': ['40.44242', '-3.69101']}

In [6]:
Madrid = cf.geocode("Castellana 81, Paseo de la Castellana, Madrid")

In [7]:
Madrid

{'type': 'Point', 'coordinates': ['40.44242', '-3.69101']}

### Airports

In [8]:
parametros_airp = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "Aeropuerto"
}

In [9]:
airports = requests.get(url_query, params = parametros_airp).json()

In [10]:
aeropuerto = airports["response"]["venues"]

In [11]:
extraer_air = cf.extraetodo(aeropuerto)

In [12]:
df_airp = pd.DataFrame(extraer_air)
df_airp.head()

,nombre,latitud,longitud,location
0,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162..."
1,Metro Madrid Línea 8 (Nuevos Ministerios - Aer...,40.445478,-3.691767,"{'type': 'Point', 'coordinates': [40.445477752..."
2,Metro Aeropuerto T1-T2-T3,40.468279,-3.571256,"{'type': 'Point', 'coordinates': [40.468278765..."
3,Transfer Aeropuerto Madrid - Transporte Coyoacan,40.420315,-3.697355,"{'type': 'Point', 'coordinates': [40.420315457..."
4,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,40.419047,-3.692954,"{'type': 'Point', 'coordinates': [40.419047439..."


In [13]:
df_airport=df_airp.drop(df_airp.index[[1,3,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]])

In [14]:
df_airport["categoria"] = "Airport"
df_airport.head()

,nombre,latitud,longitud,location,categoria
0,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162...",Airport
2,Metro Aeropuerto T1-T2-T3,40.468279,-3.571256,"{'type': 'Point', 'coordinates': [40.468278765...",Airport
4,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,40.419047,-3.692954,"{'type': 'Point', 'coordinates': [40.419047439...",Airport
5,Aeropuerto Madrid Barajas,40.432060,-3.679915,"{'type': 'Point', 'coordinates': [40.43205988,...",Airport
12,Aeropuerto Madrid-gelate,40.422891,-3.700950,"{'type': 'Point', 'coordinates': [40.422891164...",Airport


### Party

In [15]:
parametros_party = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "Discoteca"
}

In [16]:
disco = requests.get(url_query, params = parametros_party).json()

In [17]:
discoteca = disco["response"]["venues"]

In [18]:
extraer_disco = cf.extraetodo(discoteca)

In [19]:
df_disco = pd.DataFrame(extraer_disco)

In [20]:
df_disco["categoria"] = "Disco"
df_disco.head()

,nombre,latitud,longitud,location,categoria
0,Discoteca Nells Madrid,40.439664,-3.682127,"{'type': 'Point', 'coordinates': [40.439664, -...",Disco
1,Discoteca Palace,40.418451,-3.709102,"{'type': 'Point', 'coordinates': [40.418450979...",Disco
2,Discoteca Almontes Sala Rociera,40.432799,-3.678725,"{'type': 'Point', 'coordinates': [40.43279897,...",Disco
3,Discoteca Marmara,40.457896,-3.685939,"{'type': 'Point', 'coordinates': [40.457895509...",Disco
4,la cubana discoteca,40.450147,-3.695196,"{'type': 'Point', 'coordinates': [40.450147, -...",Disco


### Vegan Restaurants

In [21]:
parametros_vegan = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "vegan"
}

In [22]:
vegan = requests.get(url_query, params = parametros_vegan).json()

In [23]:
vegano = vegan["response"]["venues"]

In [24]:
extraer_vegan = cf.extraetodo(vegano)

In [25]:
df_vegan = pd.DataFrame(extraer_vegan)
len(df_vegan)

30

In [26]:
df_vegans=df_vegan.drop(df_vegan.index[[15,18]])

In [27]:
df_vegans["categoria"] = "Vegan"
df_vegans.head()

,nombre,latitud,longitud,location,categoria
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,"{'type': 'Point', 'coordinates': [40.4424564, ...",Vegan
1,Vegan Streets Madrid Centro,40.417512,-3.697636,"{'type': 'Point', 'coordinates': [40.4175117, ...",Vegan
2,vegetarian / vegan restaurant,40.432974,-3.686693,"{'type': 'Point', 'coordinates': [40.432974, -...",Vegan
3,Mad Mad Vegan,40.424183,-3.700731,"{'type': 'Point', 'coordinates': [40.424183, -...",Vegan
4,Los Andenes Vegan,40.435206,-3.710348,"{'type': 'Point', 'coordinates': [40.435206, -...",Vegan


### Nursery

In [28]:
parametros_nurs = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "guarderías"
}

In [29]:
nursery = requests.get(url_query, params = parametros_nurs).json()

In [30]:
guarderia = nursery["response"]["venues"]

In [31]:
extraer_nursery = cf.extraetodo(guarderia)

In [32]:
df_nursery = pd.DataFrame(extraer_nursery)
df_nursery["categoria"] = "Nursery"
df_nursery

,nombre,latitud,longitud,location,categoria
0,Guarderia San Nicolas,40.421913,-3.668392,"{'type': 'Point', 'coordinates': [40.421913, -...",Nursery
1,Guardería San Roque,40.389522,-3.736510,"{'type': 'Point', 'coordinates': [40.389521950...",Nursery
2,guarderia Santa Catalina,40.458316,-3.779061,"{'type': 'Point', 'coordinates': [40.458316239...",Nursery
3,Guarderia santa isabel,40.542532,-3.637349,"{'type': 'Point', 'coordinates': [40.542531541...",Nursery


### Unión de los 4 dataframes de requerimientos para Madrid

In [145]:
MADRID = cf.suma_requisitos(df_airport,df_disco,df_vegans,df_nursery)
MADRID

,nombre,latitud,longitud,location,categoria
0,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162...",Airport
2,Metro Aeropuerto T1-T2-T3,40.468279,-3.571256,"{'type': 'Point', 'coordinates': [40.468278765...",Airport
4,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,40.419047,-3.692954,"{'type': 'Point', 'coordinates': [40.419047439...",Airport
5,Aeropuerto Madrid Barajas,40.432060,-3.679915,"{'type': 'Point', 'coordinates': [40.43205988,...",Airport
12,Aeropuerto Madrid-gelate,40.422891,-3.700950,"{'type': 'Point', 'coordinates': [40.422891164...",Airport
...,...,...,...,...,...
29,Encuentro Vegano,40.406691,-3.639279,"{'type': 'Point', 'coordinates': [40.406691, -...",Vegan
0,Guarderia San Nicolas,40.421913,-3.668392,"{'type': 'Point', 'coordinates': [40.421913, -...",Nursery
1,Guardería San Roque,40.389522,-3.736510,"{'type': 'Point', 'coordinates': [40.389521950...",Nursery
2,guarderia Santa Catalina,40.458316,-3.779061,"{'type': 'Point', 'coordinates': [40.458316239...",Nursery


In [34]:
MADRID["ciudad"] = "Madrid"

In [35]:
x = MADRID.to_dict(orient = "records")

In [36]:
client = MongoClient("localhost:27017")
db = client.get_database("PROYECTO3_GEOQ")#me conecto a la base de datos creada en Mongo con toda la info de Madrid
madrid = db["MADRID"] #creo nueva coleccion en mi BBDD, la guardo en variable para trabajar con ella
madrid.insert_many(x) #inserto directamente en Mongo mi colección, sin tener que crearla desde Mongo

In [37]:
conect_madrid = db.get_collection("MADRID") #me conecto a la colección de Madrid de la BBDD de Mongo, la re

In [38]:
index_madrid = db.MADRID.create_index([("location", GEOSPHERE)]) #creo un índice

In [39]:
Madrid_bien = cf.type_point ([40.44242,-3.69101])

In [40]:
query=[{
"$geoNear": {'near': [40.44242, -3.69101],
             'distanceField': 'distance',
             'maxDistance': 10000,
             'distanceMultiplier': 6371,
             'spherical'  : True}}]

geoloc = conect_madrid.aggregate(query)
geo_madrid_json = json.loads(dumps(geoloc))

In [41]:
df_Madrid_total = pd.DataFrame(geo_madrid_json)
df_Madrid_total.drop_duplicates(["latitud", "longitud"], inplace = True)
df_Madrid_total.head()

,_id,nombre,latitud,longitud,location,categoria,ciudad,distance
0,{'$oid': '619284edad1b9302deb96006'},La Nuit Summertime,40.448909,-3.695329,"{'type': 'Point', 'coordinates': [40.448908968...",Disco,Madrid,0.865509
12,{'$oid': '6192ac13c39328a44876c650'},la cubana discoteca,40.450147,-3.695196,"{'type': 'Point', 'coordinates': [40.450147, -...",Disco,Madrid,0.975614
24,{'$oid': '619284edad1b9302deb95feb'},Discoteca Nells Madrid,40.439664,-3.682127,"{'type': 'Point', 'coordinates': [40.439664, -...",Disco,Madrid,1.034004
36,{'$oid': '619284edad1b9302deb9600b'},vegetarian / vegan restaurant,40.432974,-3.686693,"{'type': 'Point', 'coordinates': [40.432974, -...",Vegan,Madrid,1.152862
48,{'$oid': '6192bad39e25b2b2f609578f'},Restaurante Macadamia - Discoteca Graf,40.437559,-3.681101,"{'type': 'Point', 'coordinates': [40.437559354...",Disco,Madrid,1.226803


In [42]:
df_Madrid_distance = pd.DataFrame(df_Madrid_total.groupby(["categoria"])["distance"].mean()).reset_index()
df_Madrid_distance

,categoria,distance
0,Airport,8.075288
1,Disco,4.889466
2,Nursery,8.424857
3,Vegan,4.420424


In [43]:
geoloc #transformar un cursor en json

### Normalizamos datos

In [44]:
normalizado = cf.normalizacion(df_Madrid_distance,"distance")

In [45]:
normalizado

[0.9127044506782767, 0.11713081151493125, 1.0, 0.0]

In [46]:
df_Madrid_distance["Normalizado"] = cf.normalizacion(df_Madrid_distance,"distance")

In [47]:
df_Madrid_distance

,categoria,distance,Normalizado
0,Airport,8.075288,0.912704
1,Disco,4.889466,0.117131
2,Nursery,8.424857,1.000000
3,Vegan,4.420424,0.000000


In [48]:
df_Madrid_distance["Ponderado"] = cf.pesos(df_Madrid_distance)

In [49]:
df_Madrid_distance["Ciudad"] = "Madrid"
df_Madrid_distance

,categoria,distance,Normalizado,Ponderado,Ciudad
0,Airport,8.075288,0.912704,0.365082,Madrid
1,Disco,4.889466,0.117131,0.023426,Madrid
2,Nursery,8.424857,1.000000,0.300000,Madrid
3,Vegan,4.420424,0.000000,0.000000,Madrid


## MALLORCA

In [50]:
Palma = cf.geocode("P.º de Mallorca, 7, 07011 Palma, Islas Baleares")

In [51]:
Palma

{'type': 'Point', 'coordinates': ['39.57355', '2.64177']}

In [52]:
Palma_bien = cf.type_point ([39.57355,2.64177])

### Airports

In [53]:
parametros_airp_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "Aeropuerto"
}

In [54]:
airports_p = requests.get(url_query, params = parametros_airp_p).json()

In [55]:
aeropuerto_p = airports_p["response"]["venues"]

In [56]:
extraer_air_p = cf.extraetodo(aeropuerto_p)

In [57]:
df_air_p = pd.DataFrame(extraer_air_p)
df_air_p
df_air_palma= df_air_p.head(1)

In [58]:
df_air_palma["categoria"] = "Airport"

<ipython-input-58-02355b314cbf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air_palma["categoria"] = "Airport"


In [59]:
df_air_palma

,nombre,latitud,longitud,location,categoria
0,Aeropuerto de Mallorca,39.547252,2.730011,"{'type': 'Point', 'coordinates': [39.547252019...",Airport


### Party

In [60]:
parametros_party_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "pub"
}

In [61]:
disco_p = requests.get(url_query, params = parametros_party_p).json()

In [62]:
discoteca_p = disco_p["response"]["venues"]

In [63]:
extraer_disco_p = cf.extraetodo(discoteca_p)

In [64]:
df_party_p = pd.DataFrame(extraer_disco_p)

In [65]:
df_party_palma=df_party_p.iloc[0:16]
df_party_palma["categoria"] = "Disco"

<ipython-input-65-619a3c9d3574>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_party_palma["categoria"] = "Disco"


### Vegan Restaurants

In [66]:
parametros_vegan_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "vegan"
}

In [67]:
vegan_p = requests.get(url_query, params = parametros_vegan_p).json()

In [68]:
vegano_p = vegan_p["response"]["venues"]

In [69]:
extraer_vegan_p = cf.extraetodo(vegano_p)

In [70]:
df_vegan_p = pd.DataFrame(extraer_vegan_p)

In [71]:
df_vegan_palma = df_vegan_p.drop(df_vegan_p.index[[4,8]])
df_vegan_palma["categoria"] = "Vegan"

### Nursery

In [72]:
parametros_nurs_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "escoleta infantil"
}

In [73]:
nursery_p = requests.get(url_query, params = parametros_nurs_p).json()

In [74]:
guarderia_p = nursery_p["response"]["venues"]

In [75]:
extraer_nursery_p = cf.extraetodo(guarderia_p)

In [76]:
df_nursery_p = pd.DataFrame(extraer_nursery_p)
df_nursery_p["categoria"] = "Nursery"

### Unión de los 4 dataframes de requerimientos para Mallorca

In [77]:
MALLORCA = cf.suma_requisitos(df_air_palma,df_party_palma,df_vegan_palma,df_nursery_p)

In [78]:
MALLORCA["ciudad"] = "Mallorca"

In [79]:
y = MALLORCA.to_dict(orient = "records")

In [80]:
client = MongoClient("localhost:27017")
db = client.get_database("PROYECTO3_GEOQ")#me conecto a la base de datos creada en Mongo con toda la info de Madrid
madrid = db["MALLORCA"] #creo nueva coleccion en mi BBDD, la guardo en variable para trabajar con ella
madrid.insert_many(y)

In [81]:
conect_mallorca = db.get_collection("MALLORCA")

In [82]:
index_mallorca = db.MALLORCA.create_index([("location", GEOSPHERE)])

In [83]:
query1=[{
"$geoNear": {'near': [39.57355, 2.64177],
             'distanceField': 'distance',
             'maxDistance': 10000,
             'distanceMultiplier': 6371,
             'spherical'  : True}}]

geoloc = conect_mallorca.aggregate(query1)
geo_mallorca_json = json.loads(dumps(geoloc))

In [84]:
df_Mallorca_total = pd.DataFrame(geo_mallorca_json)
df_Mallorca_total.drop_duplicates(["latitud", "longitud"], inplace = True)
df_Mallorca_total.head()

,_id,nombre,latitud,longitud,location,categoria,ciudad,distance
0,{'$oid': '61928f202ce356a22f7eab31'},Pub Micros,39.574141,2.640634,"{'type': 'Point', 'coordinates': [39.574140750...",Disco,Mallorca,0.142367
8,{'$oid': '6192bae69e25b2b2f60957ca'},El Pub del Click,39.572227,2.641946,"{'type': 'Point', 'coordinates': [39.572226781...",Disco,Mallorca,0.148280
16,{'$oid': '61928f202ce356a22f7eab33'},Pub Indigno,39.574172,2.640494,"{'type': 'Point', 'coordinates': [39.574172, 2...",Disco,Mallorca,0.157812
24,{'$oid': '6192be76699639134fb13878'},Pub Corner,39.572783,2.640450,"{'type': 'Point', 'coordinates': [39.572783296...",Disco,Mallorca,0.169738
32,{'$oid': '6192bae69e25b2b2f60957c9'},Pub Chapeau,39.573509,2.643542,"{'type': 'Point', 'coordinates': [39.573509311...",Disco,Mallorca,0.197121


In [85]:
df_Mallorca_distance = pd.DataFrame(df_Mallorca_total.groupby(["categoria"])["distance"].mean()).reset_index()
df_Mallorca_distance

,categoria,distance
0,Airport,10.237544
1,Disco,2.403194
2,Nursery,2.748813
3,Vegan,5.105878


### Normalizamos datos

In [86]:
normalizado_p = cf.normalizacion(df_Mallorca_distance,"distance")

In [87]:
df_Mallorca_distance["Normalizado"] = cf.normalizacion(df_Mallorca_distance,"distance")

In [88]:
df_Mallorca_distance["Ponderado"] = cf.pesos(df_Mallorca_distance)

In [89]:
df_Mallorca_distance["Ciudad"] = "Mallorca"
df_Mallorca_distance

,categoria,distance,Normalizado,Ponderado,Ciudad
0,Airport,10.237544,1.000000,0.400000,Mallorca
1,Disco,2.403194,0.000000,0.000000,Mallorca
2,Nursery,2.748813,0.044116,0.013235,Mallorca
3,Vegan,5.105878,0.344979,0.034498,Mallorca


## SALAMANCA

In [90]:
Salamanca = cf.geocode("C. Toro, 22, 37002 Salamanca")

In [91]:
Salamanca

{'type': 'Point', 'coordinates': ['40.96636', '-5.66301']}

### Airports

In [92]:
parametros_airp_s = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Salamanca['coordinates'][0]}, {Salamanca['coordinates'][1]}",
    "query": "Aeropuerto"
}

In [93]:
airports_s = requests.get(url_query, params = parametros_airp_s).json()

In [94]:
airports_s["response"]["venues"][0]["name"]

'Aeropuerto de Salamanca-Matacán (SLM)'

In [95]:
aeropuerto_s = airports_s["response"]["venues"]

In [96]:
extraer_air_s = cf.extraetodo(aeropuerto_s)

In [97]:
df_air_s = pd.DataFrame(extraer_air_s)
df_air_s["categoria"] = "Airport"

### Party

In [98]:
parametros_party_s = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Salamanca['coordinates'][0]}, {Salamanca['coordinates'][1]}",
    "query": "Pub"
}

In [99]:
disco_s = requests.get(url_query, params = parametros_party_s).json()

In [100]:
discoteca_s = disco_s["response"]["venues"]

In [101]:
extraer_disco_s = cf.extraetodo(discoteca_s)

In [102]:
df_disco_s = pd.DataFrame(extraer_disco_s)

In [103]:
df_party_salamanca=df_disco_s.iloc[0:15]
df_party_salamanca["categoria"] = "Disco"

<ipython-input-103-3a80732eb150>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_party_salamanca["categoria"] = "Disco"


### Vegan Restaurants

In [104]:
parametros_vegan_s = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Salamanca['coordinates'][0]}, {Salamanca['coordinates'][1]}",
    "query": "vegan"
}

In [105]:
vegan_s = requests.get(url_query, params = parametros_vegan_s).json()

In [106]:
vegano_s = vegan_s["response"]["venues"]

In [107]:
extraer_vegan_s = cf.extraetodo(vegano_s)

In [108]:
df_vegan_s = pd.DataFrame(extraer_vegan_s)
df_vegan_s["categoria"] = "Vegan"

### Nursery

In [109]:
parametros_nurs_s = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Salamanca['coordinates'][0]}, {Salamanca['coordinates'][1]}",
    "query": "guarderia"
}

In [110]:
nursery_s = requests.get(url_query, params = parametros_nurs_s).json()

In [111]:
guarderia_s = nursery_s["response"]["venues"]

In [112]:
extraer_nursery_s = cf.extraetodo(guarderia_s)

In [113]:
df_nursery_s = pd.DataFrame(extraer_nursery_s)
df_nursery_s["categoria"] = "Nursery"

### Unión de los 4 dataframes de requerimientos para Salamanca

In [114]:
SALAMANCA = cf.suma_requisitos(df_air_s,df_party_salamanca,df_vegan_s,df_nursery_s)

In [115]:
SALAMANCA["ciudad"] = "Salamanca"

In [116]:
z = SALAMANCA.to_dict(orient = "records")

In [117]:
client = MongoClient("localhost:27017")
db = client.get_database("PROYECTO3_GEOQ")#me conecto a la base de datos creada en Mongo con toda la info de Madrid
madrid = db["SALAMANCA"] #creo nueva coleccion en mi BBDD, la guardo en variable para trabajar con ella
madrid.insert_many(z)

In [118]:
conect_salamanca = db.get_collection("SALAMANCA")

In [119]:
index_salamanca = db.SALAMANCA.create_index([("location", GEOSPHERE)])

In [120]:
Salamanca

{'type': 'Point', 'coordinates': ['40.96636', '-5.66301']}

In [121]:
Salamanca_bien = cf.type_point ([40.96636,-5.66301])

In [122]:
query3=[{
"$geoNear": {'near': [40.96636, -5.66301],
             'distanceField': 'distance',
             'maxDistance': 10000,
             'distanceMultiplier': 6371,
             'spherical'  : True}}]

geoloc = conect_salamanca.aggregate(query3)
geo_salamanca_json = json.loads(dumps(geoloc))

In [123]:
df_Salamanca_total = pd.DataFrame(geo_salamanca_json)
df_Salamanca_total.drop_duplicates(["latitud", "longitud"], inplace = True)
df_Salamanca_total.head()

,_id,nombre,latitud,longitud,location,distance,categoria,ciudad
0,{'$oid': '61925b78a7dc67c8a00c466e'},King's Pub,40.967073,-5.663044,"{'type': 'Point', 'coordinates': [40.967073, -...",0.078986,NaN,NaN
13,{'$oid': '61925b78a7dc67c8a00c466f'},Tee Pub,40.967113,-5.662745,"{'type': 'Point', 'coordinates': [40.967113010...",0.088393,NaN,NaN
26,{'$oid': '6192b228cd58beb590d45d5b'},O'haras's irish pub,40.966010,-5.663938,"{'type': 'Point', 'coordinates': [40.96601, -5...",0.110217,NaN,Salamanca
39,{'$oid': '6192b335cd58beb590d45d9c'},Oharas irish pub,40.965856,-5.664116,"{'type': 'Point', 'coordinates': [40.965855897...",0.135041,Disco,Salamanca
52,{'$oid': '6192bb049e25b2b2f6095801'},The Liverpool Pub,40.964024,-5.662056,"{'type': 'Point', 'coordinates': [40.964023643...",0.279425,Disco,Salamanca


In [124]:
df_Salamanca_distance = pd.DataFrame(df_Salamanca_total.groupby(["categoria"])["distance"].mean()).reset_index()
df_Salamanca_distance

,categoria,distance
0,Disco,0.477305
1,Nursery,0.649408
2,Vegan,0.340160


### Normalizamos datos

In [125]:
normalizado_s = cf.normalizacion(df_Salamanca_distance,"distance")

In [126]:
df_Salamanca_distance["Normalizado"] = cf.normalizacion(df_Salamanca_distance,"distance")

In [127]:
df_Salamanca_distance["Ponderado"] = cf.pesos(df_Salamanca_distance)

In [128]:
df_Salamanca_distance["Ciudad"] = "Salamanca"

In [129]:
df_Salamanca_distance

,categoria,distance,Normalizado,Ponderado,Ciudad
0,Disco,0.477305,0.44348,0.088696,Salamanca
1,Nursery,0.649408,1.00000,0.300000,Salamanca
2,Vegan,0.340160,0.00000,0.000000,Salamanca


In [131]:
TOTAL = cf.suma_final(df_Madrid_distance,df_Mallorca_distance,df_Salamanca_distance)

In [132]:
TOTAL

,categoria,distance,Normalizado,Ponderado,Ciudad
0,Airport,8.075288,0.912704,0.365082,Madrid
1,Disco,4.889466,0.117131,0.023426,Madrid
2,Nursery,8.424857,1.000000,0.300000,Madrid
3,Vegan,4.420424,0.000000,0.000000,Madrid
0,Airport,10.237544,1.000000,0.400000,Mallorca
1,Disco,2.403194,0.000000,0.000000,Mallorca
2,Nursery,2.748813,0.044116,0.013235,Mallorca
3,Vegan,5.105878,0.344979,0.034498,Mallorca
0,Disco,0.477305,0.443480,0.088696,Salamanca
1,Nursery,0.649408,1.000000,0.300000,Salamanca


In [134]:
FINAL = TOTAL.groupby(['Ciudad'])['Ponderado'].sum()

In [137]:
TOTAL_FINAL = pd.DataFrame(FINAL).reset_index()

In [138]:
TOTAL_FINAL

,Ciudad,Ponderado
0,Madrid,0.688508
1,Mallorca,0.447733
2,Salamanca,0.388696


In [139]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [140]:
inicial_lat = 40.44242
inicial_long = -3.69101

In [147]:
map_madrid= Map(location = [40.44242,-3.69101], zoom_start = 15)

In [154]:
for i,row in MADRID.iterrows():
    categoria = {"location": [row["latitud"], row["longitud"]], "tooltip": row["categoria"]}
    
    if row["categoria"] == "Airport":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="paper-plane",
                     icon_color="black")
        
    elif row["categoria"] == "Vegan":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="cutlery",
                     icon_color="black")
        
    elif row["categoria"] == "Nursery":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="soccer-ball-o",
                     icon_color="black")
        
    elif row["categoria"] == "Disco":
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
        
    mark = Marker(**categoria, icon=icono)
    mark.add_to(map_madrid)
map_madrid

In [148]:
map_mallorca= Map(location = [39.57355,2.64177], zoom_start = 15)

In [156]:
for i,row in MALLORCA.iterrows():
    categoria = {"location": [row["latitud"], row["longitud"]], "tooltip": row["categoria"]}
    
    if row["categoria"] == "Airport":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="paper-plane",
                     icon_color="black")
        
    elif row["categoria"] == "Vegan":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="cutlery",
                     icon_color="black")
        
    elif row["categoria"] == "Nursery":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="soccer-ball-o",
                     icon_color="black")
        
    elif row["categoria"] == "Disco":
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
        
    mark = Marker(**categoria, icon=icono)
    mark.add_to(map_mallorca)
map_mallorca

In [150]:
map_salamanca= Map(location = [40.96636,-5.66301], zoom_start = 15)

In [155]:
for i,row in SALAMANCA.iterrows():
    categoria = {"location": [row["latitud"], row["longitud"]], "tooltip": row["categoria"]}
    
    if row["categoria"] == "Airport":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="paper-plane",
                     icon_color="black")
        
    elif row["categoria"] == "Vegan":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="cutlery",
                     icon_color="black")
        
    elif row["categoria"] == "Nursery":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="soccer-ball-o",
                     icon_color="black")
        
    elif row["categoria"] == "Disco":
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
        
    mark = Marker(**categoria, icon=icono)
    mark.add_to(map_salamanca)
map_salamanca